# One Sensor

In [4]:
import time
import signal
import sys
import numpy as np
from hmmlearn import hmm
import RPi.GPIO as GPIO

from ring_buffer import RingBuffer

DEBUG = True

SPEED_OF_SOUND = 13503.9 # inches per second

GPIO_TRIGGER = 4
GPIO_ECHO = 17

THRESHOLD = 40

class UltrasonicSensor:
    def __init__(self, trigger, echo):
        self.trigger = trigger
        self.echo = echo

        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.trigger, GPIO.OUT)
        GPIO.setup(self.echo, GPIO.IN)

    def distance(self):
        GPIO.output(self.trigger, True)
        start_time = time.perf_counter()
        GPIO.output(self.trigger, False)

        GPIO.wait_for_edge(self.echo, GPIO.FALLING, timeout=100)
        end_time = time.perf_counter()

        elapsed = end_time - start_time

        round_trip_distance = elapsed * SPEED_OF_SOUND
        return round_trip_distance / 2

    def sample(self, poll_time, count=1000):
        history = RingBuffer(capacity=20, dtype=np.float)
        avg = []
        for _ in range(count):
            dist = self.distance()
            history.append(dist)

            avg.append(np.median(history))
            
            time.sleep(poll_time)
            
        return avg

In [27]:
model = hmm.GaussianHMM(
            n_components=2,
            covariance_type="full",
            startprob_prior=0.0,
        )

In [6]:
sensor = UltrasonicSensor(
    trigger=GPIO_TRIGGER,
    echo=GPIO_ECHO,
)

In [13]:
samples = sensor.sample(poll_time=0.05, count=1000)
samples_r = np.reshape(samples, (-1, 1))
samples

[38.85161822786322,
 38.566068109912159,
 38.280517991961098,
 38.309706642538366,
 38.280517991961098,
 38.273661645008545,
 38.266805298055985,
 38.21599334330385,
 38.165181388551716,
 38.154280325502626,
 38.143379262453529,
 38.153223704094124,
 38.143379262453529,
 38.113133990467759,
 38.082888718481989,
 38.113133990467759,
 38.082888718481989,
 38.039986784476781,
 38.004822534046248,
 38.043855626264119,
 38.043855626264119,
 38.002884576015639,
 38.002884576015639,
 37.999015734228308,
 37.97950272525631,
 37.999015734228308,
 37.999015734228308,
 37.984951684720002,
 37.999015734228308,
 37.984951684720002,
 37.984951684720002,
 37.984951684720002,
 37.984070348097887,
 37.984070348097887,
 37.993742452566224,
 37.982490623436753,
 37.993742452566224,
 37.993742452566224,
 37.993742452566224,
 37.982490623436753,
 37.982490623436753,
 37.982490623436753,
 37.982490623436753,
 37.982490623436753,
 37.982490623436753,
 37.971586023250708,
 37.981258127719045,
 37.992159190768

In [28]:
model.fit(samples_r)
predictions = model.predict(samples_r)
predictions

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWa

ValueError: array must not contain infs or NaNs

In [18]:
list(zip(samples, predictions))

[(38.85161822786322, 1),
 (38.566068109912159, 1),
 (38.280517991961098, 1),
 (38.309706642538366, 1),
 (38.280517991961098, 1),
 (38.273661645008545, 1),
 (38.266805298055985, 1),
 (38.21599334330385, 1),
 (38.165181388551716, 1),
 (38.154280325502626, 1),
 (38.143379262453529, 1),
 (38.153223704094124, 1),
 (38.143379262453529, 1),
 (38.113133990467759, 1),
 (38.082888718481989, 1),
 (38.113133990467759, 1),
 (38.082888718481989, 1),
 (38.039986784476781, 0),
 (38.004822534046248, 0),
 (38.043855626264119, 0),
 (38.043855626264119, 0),
 (38.002884576015639, 0),
 (38.002884576015639, 0),
 (37.999015734228308, 0),
 (37.97950272525631, 0),
 (37.999015734228308, 0),
 (37.999015734228308, 0),
 (37.984951684720002, 0),
 (37.999015734228308, 0),
 (37.984951684720002, 0),
 (37.984951684720002, 0),
 (37.984951684720002, 0),
 (37.984070348097887, 0),
 (37.984070348097887, 0),
 (37.993742452566224, 0),
 (37.982490623436753, 0),
 (37.993742452566224, 0),
 (37.993742452566224, 0),
 (37.9937424525

In [19]:
model.score(samples_r)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


417.56404822309224

In [47]:
GPIO.cleanup()

In [77]:
model = hmm.GaussianHMM(
            n_components=2,
            covariance_type="full",
        )
model.fit(samples_r)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWa

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=2, n_iter=10, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [78]:
model.predict(samples_r)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [80]:
list(zip(_78, samples_r))

[(0, array([ 91.78398263])),
 (0, array([ 90.67114314])),
 (0, array([ 90.43781944])),
 (0, array([ 90.33530791])),
 (0, array([ 90.23279637])),
 (0, array([ 90.33530791])),
 (0, array([ 90.23279637])),
 (0, array([ 89.98223836])),
 (0, array([ 89.73168036])),
 (0, array([ 89.98223836])),
 (0, array([ 89.73168036])),
 (0, array([ 89.72482362])),
 (0, array([ 89.71796688])),
 (0, array([ 89.71567805])),
 (0, array([ 89.71338923])),
 (0, array([ 89.67453172])),
 (0, array([ 89.63567421])),
 (0, array([ 89.59698893])),
 (0, array([ 89.63567421])),
 (0, array([ 89.63883749])),
 (0, array([ 89.63883749])),
 (0, array([ 89.677695])),
 (0, array([ 89.677695])),
 (0, array([ 89.677695])),
 (0, array([ 89.67998382])),
 (0, array([ 89.67998382])),
 (0, array([ 89.67998382])),
 (0, array([ 89.83119387])),
 (0, array([ 90.16597583])),
 (0, array([ 90.16597583])),
 (0, array([ 90.93770001])),
 (1, array([ 92.73997436])),
 (1, array([ 98.76079631])),
 (1, array([ 103.68847071])),
 (1, array([ 103.68